In [1]:
import numpy as np
import pandas as pd
import tables

In [2]:
INPUT_DIR = '../../input/m5-forecasting-accuracy/'
OUTPUT_DIR =  '../../input/lv3/'

In [3]:
price_df = pd.read_csv(INPUT_DIR+'sell_prices.csv')
cal_df = pd.read_csv(INPUT_DIR+'calendar.csv')

In [4]:
p_df = pd.merge(price_df, cal_df.loc[:,['wm_yr_wk', 'd']], on='wm_yr_wk')

In [5]:
p_df

,store_id,item_id,wm_yr_wk,sell_price,d
0,CA_1,HOBBIES_1_001,11325,9.58,d_897
1,CA_1,HOBBIES_1_001,11325,9.58,d_898
2,CA_1,HOBBIES_1_001,11325,9.58,d_899
3,CA_1,HOBBIES_1_001,11325,9.58,d_900
4,CA_1,HOBBIES_1_001,11325,9.58,d_901
...,...,...,...,...,...
47735392,WI_3,FOODS_3_825,11105,4.00,d_31
47735393,WI_3,FOODS_3_825,11105,4.00,d_32
47735394,WI_3,FOODS_3_825,11105,4.00,d_33
47735395,WI_3,FOODS_3_825,11105,4.00,d_34


In [6]:
store_list = p_df['store_id'].unique().tolist()

In [7]:
for store in store_list:
    sum_df = p_df[p_df['store_id']==store].groupby(["d"])["sell_price"].agg(['count', 'sum'])
    price_fea = pd.merge(cal_df.loc[:,['d']],sum_df, on='d').T
    price_fea = price_fea.reset_index()
    price_fea.to_csv(OUTPUT_DIR+f'price_fea_{store}.csv',index=False)

In [ ]:
# sales feature

In [13]:
sales = pd.read_csv(INPUT_DIR+'sales_train_evaluation.csv')

In [9]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [14]:
sales = sales.drop(['d_332','d_697','d_1062','d_1427','d_1792'],axis=1)

In [15]:
import matplotlib.pyplot as plt

In [16]:
for store in store_list:
    sum_sales_1shift = np.array(sales[sales['store_id']==store].iloc[:,6:-1]).sum(axis=0)
    sum_sales = np.array(sales.iloc[:,7:]).sum(axis=0)
    diff_1 = sum_sales - sum_sales_1shift
    diff_1_diff_1 = diff_1[1:] - diff_1[:-1]
    np.save(OUTPUT_DIR+f'sales_diff_{store}.npy', np.concatenate([diff_1[1:].reshape(1,-1),diff_1_diff_1.reshape(1,-1)],axis=0))

In [ ]:
# 週ごとの最大値・最小値を追加

In [17]:
cal_df = pd.read_csv(INPUT_DIR+'calendar.csv')
sales = pd.read_csv(INPUT_DIR+'sales_train_evaluation.csv')

In [28]:
cal_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [29]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [18]:
for store in store_list:
    sales_sum = pd.DataFrame(np.array(sales[sales['store_id']==store].iloc[:,6:]).sum(axis=0).reshape(1,-1),columns=[f'd_{i+1}' for i in range(1941)],index=['sales_sum'])
    sales_sum = sales_sum.T.reset_index().rename(columns={'index': 'd'})
    test_df = pd.merge(cal_df.loc[:,['d','wm_yr_wk']],sales_sum, on='d')
    min_max = pd.merge(test_df, test_df.groupby('wm_yr_wk')['sales_sum'].agg({max,min}).reset_index(),on='wm_yr_wk').loc[:,['d','max','min']].T
    min_max.reset_index().to_csv(OUTPUT_DIR+f'min_max_{store}.csv')

In [ ]:
# sales lv3

In [82]:
sales = pd.read_csv(INPUT_DIR+'sales_train_evaluation.csv')

In [27]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [83]:
columns = ['store_id']+[f'd_{i+1}' for i in range(1941)]
sales = sales.loc[:,columns]

In [32]:
sales

,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,CA_1,0,0,0,0,0,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,CA_1,0,0,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,CA_1,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,CA_1,0,0,0,0,0,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,CA_1,0,0,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,WI_3,0,0,2,2,0,3,1,4,1,...,1,0,3,0,1,1,0,0,1,1
30486,WI_3,0,0,0,0,0,5,0,1,1,...,0,0,0,0,0,0,1,0,1,0
30487,WI_3,0,6,0,2,2,4,1,8,5,...,0,0,1,2,0,1,0,1,0,2
30488,WI_3,0,0,0,0,0,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [80]:
for store in store_list:
    sales_sum = pd.DataFrame(np.array(sales[sales['store_id']==store].iloc[:,1:].sum(axis=0)).reshape(1,-1),columns=[f'd_{i+1}' for i in range(1941)],index=['sales_sum'])
    sales_sum = sales_sum.T.reset_index().rename(columns={'index': 'd'}).T
    sales_sum.reset_index().to_csv(OUTPUT_DIR+f'sales_lv3_{store}.csv')

In [81]:
store

'WI_3'

In [88]:
sales.store_id.unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)